# Wrangling

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
vancouver_crime_data <- read.csv("crimedata_csv_AllNeighbourhoods_AllYears.csv")

In [4]:
head(vancouver_crime_data)

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,Other Theft,2019,3,14,10,34,47XX OAK ST,South Cambie,490706.9,5454474
2,Other Theft,2005,8,1,0,0,47XX OAK ST,South Cambie,490706.9,5454474
3,Other Theft,2012,12,6,9,30,47XX OAK ST,South Cambie,490706.9,5454474
4,Other Theft,2004,1,25,17,0,47XX OAK ST,South Cambie,490710.6,5454535
5,Other Theft,2020,9,22,11,36,47XX OSLER ST,Shaughnessy,490528.5,5454450
6,Other Theft,2003,8,18,7,30,47XX OSLER ST,Shaughnessy,490529.8,5454502


In [15]:
vancouver_assualts <- vancouver_crime_data %>%
    filter(TYPE == "Theft" | TYPE == "" | TYPE == "" | TYPE == "")

In [16]:
vancouver_assualts

TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
Offence Against a Person,2004,8,17,0,0,OFFSET TO PROTECT PRIVACY,West End,0,0
Offence Against a Person,2005,4,20,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2006,10,7,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2005,3,25,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2005,3,25,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2003,12,6,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2003,7,22,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2008,5,28,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
Offence Against a Person,2010,10,31,0,0,OFFSET TO PROTECT PRIVACY,Central Business District,0,0
